In [2]:
%load_ext autoreload
%autoreload 2

In [30]:
import pandas as pd
import data
from IPython.display import HTML
import myWidgets
import logisticRegression
import numpy as np
from sklearn.metrics import confusion_matrix, precision_recall_curve, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import Model
import ModelTracker

df = data.get_data()
tracker = ModelTracker.ModelTracker()

Le but de ce projet est de construire un modèle de prédiction simple afin de me familiariser avec les concepts de base d'intelligence artificielle. 

Le projet va devoir prédire quels clients sont plus susceptibles d'acheter une assurance caravane en fonction des données provenant du CoIL 2000 Challenge. 

Les variables qui commence par M sont des variables qui donne de l'information sur la distribution de la variable dans le zipcode area du client. (0 = aucun, 9 = beaucoup). Ce n'est pas une zone!

In [ ]:
display(myWidgets.AllVarComparator(df.columns, df).get_widget())

In [4]:
corr_discrete = df.corr(method='spearman').loc[['CARAVAN']].sort_values(by='CARAVAN', axis=1, ascending=False).drop(columns=['CARAVAN'])
corr_top = corr_discrete.unstack().sort_values(ascending=False) #2d a 1d (A,B)
corr_combined = corr_discrete.style.background_gradient(cmap='coolwarm', axis=None).format(precision=4)

display(HTML(corr_combined.to_html()))


,PPERSAUT,APERSAUT,APLEZIER,PPLEZIER,PBRAND,MINKGEM,PWAPART,MKOOPKLA,AWAPART,MHKOOP,MOPLHOOG,MAUT1,MINK4575,ABRAND,ABYSTAND,MRELGE,PBYSTAND,MINK7512,MSKA,MBERHOOG,MZPART,MBERMIDD,MOPLMIDD,MGEMOMV,MGODPR,PGEZONG,AGEZONG,MSKB1,PWAOREG,AWAOREG,AFIETS,MFWEKIND,PFIETS,MBERZELF,AZEILPL,PZEILPL,MGODRK,ALEVEN,AINBOED,PINBOED,PLEVEN,PAANHANG,AAANHANG,AMOTSCO,PMOTSCO,MAUT2,MFGEKIND,MSKB2,MINK123M,MGODOV,MGEMLEEF,AWABEDR,PWABEDR,MAANTHUI,MINK3045,PBESAUT,ABESAUT,APERSONG,PPERSONG,AVRAAUT,PVRAAUT,PWERKT,AWERKT,PTRACTOR,ATRACTOR,AWALAND,PWALAND,MRELSA,MGODGE,ABROM,PBROM,MSKC,MBERARBG,MFALLEEN,MZFONDS,MBERBOER,MBERARBO,MOSTYPE,MRELOV,MSKD,MOSHOOFD,MAUT0,MHHUUR,MINKM30,MOPLLAAG
CARAVAN,0.1637,0.1495,0.1064,0.1064,0.1006,0.1002,0.0953,0.0947,0.0900,0.0799,0.0782,0.0734,0.0699,0.0695,0.0682,0.0682,0.0681,0.0632,0.0600,0.0572,0.0554,0.0410,0.0399,0.0355,0.0348,0.0337,0.0335,0.0309,0.0303,0.0303,0.0289,0.0287,0.0287,0.0266,0.0262,0.0262,0.0196,0.0193,0.0191,0.0190,0.0186,0.0146,0.0145,0.0103,0.0099,0.0084,0.0075,0.0073,0.0072,0.0041,0.0039,0.0006,0.0006,-0.0011,-0.0063,-0.0069,-0.0070,-0.0085,-0.0085,-0.0099,-0.0099,-0.0152,-0.0152,-0.0165,-0.0167,-0.0213,-0.0213,-0.0274,-0.0414,-0.0452,-0.0452,-0.0485,-0.0508,-0.0517,-0.0562,-0.0565,-0.0572,-0.0580,-0.0581,-0.0611,-0.0615,-0.0775,-0.0806,-0.0815,-0.0907


Les 10 variables les plus correlated avec la cible sont :

In [5]:
data.top_index_and_values(10 ,corr_top)

Contribution car policies                          PPERSAUT   - 0.1637
Number of car policies                             APERSAUT   - 0.1495
Number of boat policies                            APLEZIER   - 0.1064
Contribution boat policies                         PPLEZIER   - 0.1064
Contribution fire policies                         PBRAND     - 0.1006
Average income                                     MINKGEM    - 0.1002
Contribution private third party insurance see L4  PWAPART    - 0.0953
Purchasing power class                             MKOOPKLA   - 0.0947
Number of private third party insurance 1 - 12     AWAPART    - 0.0900
Home owners                                        MHKOOP     - 0.0799


In [31]:
ordinale_vars, discrete_vars = data.get_var_by_types(df)
discrete_vars.remove('CARAVAN')

X = df.drop('CARAVAN', axis=1) 
y = df['CARAVAN']  #targets

X_train, X_val, y_train, y_val = data.get_split_train_eval_data(df)

X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
X_val_np = X_val.to_numpy()
y_val_np = y_val.to_numpy()

In [32]:
w,b = logisticRegression.logistic_regression(X_train_np, y_train_np)

Iteration 0: Cost = 0.6931471805599452
Iteration 100: Cost = 0.21597549060730015
Iteration 200: Cost = 0.21097968582978055
Iteration 300: Cost = 0.20852384801501356
Iteration 400: Cost = 0.20707216607645518
Iteration 500: Cost = 0.20610886312970111
Iteration 600: Cost = 0.20541183547005737
Iteration 700: Cost = 0.20487348588058174
Iteration 800: Cost = 0.2044369993836154
Iteration 900: Cost = 0.20407009523426323


In [33]:
threshold = 0.1 #trop bas
model = Model.Model(w, b, threshold)
tracker.add(model)
model.print_stats(X_val_np, y_val_np)

Model v1 saved

Threshold:  0.1
Accuracy:  0.8180257510729614
[[916 179]
 [ 33  37]]
AUC:  0.7934377038486629


Présentement le modèle est vraiment pas bon. Même avec un threshold de 0.1, il prédit seulement 160 qui ont besoin d'une assurance caravan en plus avec un threshold de seulement 0.1

In [34]:
print(np.bincount(y_train_np))

[4378  278]


La classe est imbalanced.

Vu que la majoriter est "pas besoin d'assurance caravan", le modèle apprend probablement plutot a prédire: qui n'a pas besoin d'assurance caravan.

Donc je vais ajuster les weights pour que lorsque le modèle prédit incorrectement que quelqu'un a pas besoin d'assurance Caravan et qu'il ai tord, il sera pénaliser x fois plus. Comme ça, il va faire plus attention à eux que ceux qui n'ont pas besoin d'assurance caravan.

In [35]:
counts = np.bincount(y_train_np)
ratio = counts[0] / counts[1]

w,b = logisticRegression.logistic_regression(X_train_np, y_train_np, extra_weight=ratio)

Iteration 0: Cost = 0.6931471805599452
Iteration 100: Cost = 1.3343512549843255
Iteration 200: Cost = 1.1880672944256734
Iteration 300: Cost = 1.1375072309774037
Iteration 400: Cost = 1.11168432214414
Iteration 500: Cost = 1.093016337276002
Iteration 600: Cost = 1.0778834094949865
Iteration 700: Cost = 1.065102570866037
Iteration 800: Cost = 1.0540486751492382
Iteration 900: Cost = 1.0443170066614136


Le cout augmente, donc peut-être le learning rate est trop grand. Je vais l'ajuster.

In [36]:
counts = np.bincount(y_train_np)
ratio = counts[0] / counts[1]

learning_rate=0.005

w,b = logisticRegression.logistic_regression(X_train_np, y_train_np, extra_weight=ratio, learning_rate=learning_rate)
w_old = w
b_old = b

Iteration 0: Cost = 0.6931471805599452
Iteration 100: Cost = 0.6272215446900786
Iteration 200: Cost = 0.6121389394789053
Iteration 300: Cost = 0.6057792353705707
Iteration 400: Cost = 0.6020341046276506
Iteration 500: Cost = 0.5994261507566155
Iteration 600: Cost = 0.5974302790468196
Iteration 700: Cost = 0.595808288326045
Iteration 800: Cost = 0.594435303478852
Iteration 900: Cost = 0.5932394913544604


In [37]:
threshold = 0.1 #trop bas
model = Model.Model(w, b, threshold, improvement="adjusted weights")
tracker.add(model)

model.print_stats(X_val_np, y_val_np)

Model v2 saved

Threshold:  0.1
Accuracy:  0.07124463519313305
[[  13 1082]
 [   0   70]]
AUC:  0.7876712328767124


C'est déja mieux, mais le threshold est trop bas. Je vais essayer 0.5

In [38]:
threshold = 0.5
model = Model.Model(w, b, threshold, improvement="adjusted weights")
tracker.add(model)

model.print_stats(X_val_np, y_val_np)

Model v3 saved

Threshold:  0.5
Accuracy:  0.6824034334763949
[[742 353]
 [ 17  53]]
AUC:  0.7876712328767124


Beaucoup mieux, en plus avec un threshold de 0.5. Donc je croit, le modèle est asser précis maintenant. Je vais chercher le meilleur threshold.

In [39]:
threshold = logisticRegression.get_youden_threshold(X_val_np, y_val_np, w, b)
model = Model.Model(w, b, threshold, improvement="Youden index threshold")

YOUDEN_MODEL = "YoudenModel"
tracker.add(model, name=YOUDEN_MODEL, set_preferred=True)
model.print_stats(X_val_np, y_val_np)

Model YoudenModel saved

Threshold:  0.5287802351530482
Accuracy:  0.7167381974248928
[[782 313]
 [ 17  53]]
AUC:  0.7876712328767124


Au faite non, il n'est pas asser précis. Il a 313 False positive, donc il prédit 313 personnes comme ayant besoin d'assurance caravan en faite que non, ils ont pas besoin. En plus, il manque pas mal de personne (17) qui ont actually besoin d'une assurance caravan. 

In [40]:
data.top_index_and_values(15 ,corr_top)

Contribution car policies                          PPERSAUT   - 0.1637
Number of car policies                             APERSAUT   - 0.1495
Number of boat policies                            APLEZIER   - 0.1064
Contribution boat policies                         PPLEZIER   - 0.1064
Contribution fire policies                         PBRAND     - 0.1006
Average income                                     MINKGEM    - 0.1002
Contribution private third party insurance see L4  PWAPART    - 0.0953
Purchasing power class                             MKOOPKLA   - 0.0947
Number of private third party insurance 1 - 12     AWAPART    - 0.0900
Home owners                                        MHKOOP     - 0.0799
High level education                               MOPLHOOG   - 0.0782
1 car                                              MAUT1      - 0.0734
Income 45-75.000                                   MINK4575   - 0.0699
Number of fire policies                            ABRAND     - 0.0695
Number

On voit que PPERSAUT & APERSAUT sont les variables qui corrèles le plus avec CARAVAN, ce qui fais du sens puisque pour utiliser une Caravan, il faut au minimum une voiture pour la remorquer. 
- Donc, je pourrais indiquer à mon modèle qui si la personne a aucune voiture, alors c'est un "pas besoin d'assurance caravan" ou tout simplement les retirer du dataset. (C'est mieux de pas retirer, car sinon il pourrait miss des patterns et mal classifiés des personnes plus tard)

De plus, APLEZIER, PPLEZIER et PBRAND nous dit que les personnes qui ont des caravans sont des personnes avec un bon income, c'est surtout des clients disposant d’un pouvoir d’achat élevé.
- Donc, s'il y a moyen, peut-etre d'indiquer au modèle de faire plus attention a eux?

Je peux ajouter des "interaction terms". Pour PPERSAUT et APERSAUT c'est parfait puisque l'effet de PPERSAUT sur la probabilité d'avoir besoin d'une assurance caravan, dépend de APERSAUT (ou en d'autre mots dépend du nombre de car policies).

In [295]:
vars_to_test = [
    'PPERSAUT',   # Contribution car policies
    'APERSAUT',   # Number of car policies
    'APLEZIER',   # Number of boat policies
    'PPLEZIER',   # Contribution boat policies
    'PBRAND',     # Contribution fire policies
    'MINKGEM',    # Average income
    'PWAPART',    # Contribution private third party insurance see L4 
    'MKOOPKLA',   # Purchasing power class
    'AWAPART',    # Number of private third party insurance 1 - 12 
    'MHKOOP'      # Home owners
]

logisticRegression.interactions_terms_tester(X_train, y_train_np, w, b, vars_to_test=vars_to_test, learning_rate=0.005, ratio=ratio)

Interaction PPERSAUT_x_APERSAUT : AUC = 0.7559 (gain = +0.0010)
Interaction PPERSAUT_x_APLEZIER : AUC = 0.7606 (gain = +0.0057)
Interaction PPERSAUT_x_PPLEZIER : AUC = 0.7604 (gain = +0.0055)
Interaction PPERSAUT_x_PBRAND : AUC = 0.7599 (gain = +0.0050)
Interaction PPERSAUT_x_MINKGEM : AUC = 0.7551 (gain = +0.0002)
Interaction PPERSAUT_x_PWAPART : AUC = 0.7568 (gain = +0.0020)
Interaction PPERSAUT_x_MKOOPKLA : AUC = 0.7550 (gain = +0.0001)
Interaction PPERSAUT_x_AWAPART : AUC = 0.7560 (gain = +0.0011)
Interaction PPERSAUT_x_MHKOOP : AUC = 0.7562 (gain = +0.0013)
Interaction APERSAUT_x_APLEZIER : AUC = 0.7552 (gain = +0.0003)
Interaction APERSAUT_x_PPLEZIER : AUC = 0.7562 (gain = +0.0014)
Interaction APERSAUT_x_PBRAND : AUC = 0.7593 (gain = +0.0044)
Interaction APERSAUT_x_MINKGEM : AUC = 0.7553 (gain = +0.0004)
Interaction APERSAUT_x_PWAPART : AUC = 0.7568 (gain = +0.0019)
Interaction APERSAUT_x_MKOOPKLA : AUC = 0.7550 (gain = +0.0001)
Interaction APERSAUT_x_AWAPART : AUC = 0.7555 (gain

Je vais commencer par ajouter tout les interactions qui ont au minimum +0.003 de gain. 
Donc, PPERSAUT_x_APLEZIER, PPERSAUT_x_PPLEZIER, PPERSAUT_x_PBRAND, APERSAUT_x_PBRAND, PPLEZIER_x_MINKGEM, PPLEZIER_x_MKOOPKLA, PPLEZIER_x_MHKOOP.

In [41]:
interactions_to_add_1 = [
    ("PPERSAUT", "APLEZIER"),
    ("PPERSAUT", "PPLEZIER"),
    ("PPERSAUT", "PBRAND"),
    ("APERSAUT", "PBRAND"),
    ("APLEZIER", "MINKGEM"),
    ("APLEZIER", "MHKOOP"),
    ("PPLEZIER", "MINKGEM"),
    ("PPLEZIER", "MKOOPKLA"),
    ("PPLEZIER", "MHKOOP"),
]

X_train_final = data.add_interactions_terms(X_train, interactions_to_add_1)
X_val_final = data.add_interactions_terms(X_val, interactions_to_add_1)

X_train_final_np = X_train_final.to_numpy()
X_val_final_np = X_val_final.to_numpy()

prev_model = tracker.get_by_name(YOUDEN_MODEL)

model = Model.create_model(X_train_final_np, y_train_np, X_val_final_np, y_val_np, improvement="Added interactions terms with auc gain > 0.003", extra_weight=ratio, learning_rate=learning_rate, threshold_method="Youden")
print("With training set :")
logisticRegression.compare_auc_score(X_train_np, y_train_np, X_train_final_np, prev_model, model)

With training set :
New X : AUC = 0.7672 (gain = +0.0124)


Les intéractions ont ajoutés un gain d'AUC considérable (+1.24%)! Je vais faire un overfitting test pour voir s'il suraprenti pas.


In [42]:
X_train_interaction_1 = X_train_final
X_val_interaction_1 = X_val_final
logisticRegression.compare_model_stats(X_val_np, y_val_np, prev_model, model, X_val_final_np, isValData=True)

(Avec val data)

Old model :
Threshold:  0.5287802351530482
Accuracy:  0.7167381974248928
[[782 313]
 [ 17  53]]
AUC:  0.7876712328767124

New model :
Threshold:  0.44929478499446074
Accuracy:  0.6412017167381975
[[687 408]
 [ 10  60]]
AUC:  0.8018395303326808


<s>Sur le val data, les deux modèles détectent presque autant de vrais acheteurs. Par contre le nouveau modèle est considérablement plus précis en général. Je crois qu'il est plus avantageux de garder le nouveau modèle du au gain significatif de précision et que la perte des targets est seulement de 10.</s>

Maintenant, après avoir corrigé l'erreur d'utiliser le full training set pour évaluer et train le modèle et ensuite re-évaluer avec le test set. J'obtiens des résultats différents. Maintenant, le modèle est moins précis, mais son score AUC a augmenté d'environ 2%, donc il est plus facile pour lui de discriminer entre eux qui ont besoin d'assurance caravan vs eux que non. Malgré la perte de précision, je crois que dans ce cas si, c'est mieux de garder le nouveau modèle, car il discrimine mieux entre les targets et les non-targets.

In [43]:
INTER_TERMS_1_MODEL = "inter_terms_1" 
tracker.add(model, name=INTER_TERMS_1_MODEL, set_preferred=True)

Model inter_terms_1 saved



Maintenant, je vais voir si ajouter les interactions de termes qui ont un gain d'au moins 0.002 marche bien. Donc : PPERSAUT_x_PWAPART, APLEZIER_x_MKOOPKLA, APLEZIER_x_PPLEZIER

In [44]:
interactions_to_add_2 = [
    ('PPERSAUT', 'PWAPART'),
    ("APLEZIER", "MKOOPKLA"),
    ("APLEZIER", "PPLEZIER"),
]

X_train_final = data.add_interactions_terms(X_train_interaction_1, interactions_to_add_2)
X_train_final_np = X_train_final.to_numpy()

X_val_final = data.add_interactions_terms(X_val_interaction_1, interactions_to_add_2)
X_val_final_np = X_val_final.to_numpy()

prev_model = tracker.get_preferred()

model = Model.create_model(X_train_final_np, y_train_np, X_val_final_np, y_val_np, improvement="Added interactions terms with auc gain > 0.002", extra_weight=ratio, learning_rate=learning_rate, threshold_method="Youden")
logisticRegression.compare_auc_score(X_train_interaction_1.to_numpy(), y_train_np, X_train_final_np, prev_model, model)

New X : AUC = 0.7681 (gain = +0.0008)


In [45]:
INTER_TERMS_2_MODEL = "inter_terms_2" 
tracker.add(model, name=INTER_TERMS_2_MODEL)
logisticRegression.compare_model_stats(X_val_interaction_1.to_numpy(), y_val_np, prev_model, model, X_val_final_np, isValData=True)

Model inter_terms_2 saved

(Avec val data)

Old model :
Threshold:  0.44929478499446074
Accuracy:  0.6412017167381975
[[687 408]
 [ 10  60]]
AUC:  0.8018395303326808

New model :
Threshold:  0.45784582260954015
Accuracy:  0.6549356223175966
[[704 391]
 [ 11  59]]
AUC:  0.8017873450750164


<s> Le modèle augmente le rappel mais au cout de la précision. Je ne crois pas que sa vaut la peine, car la perte de précision est asser importante. </s>

<s>Les changements sont très faibles. Je ne vais pas ajouter les nouvelles intéractions pour éviter des complexifier le modèle inutilement. </s>

Au faites, les changements sont faibles oui, mais pour le score AUC. Si on regarde la précision, il y eu un gain asser important. Environ (+1.3% de précision). Je crois que sa vaut la peine de garder les termes additionneles pour l'instant.

In [46]:
#À garder pour plus tard au cas ou.
X_train_interaction_2 = X_train_final_np
X_val_interaction_2 = X_val_final_np

tracker.set_preferred(INTER_TERMS_2_MODEL)
model.print_stats(X_val_final_np, y_val_np)

Model inter_terms_2 is now preferred.
Threshold:  0.45784582260954015
Accuracy:  0.6549356223175966
[[704 391]
 [ 11  59]]
AUC:  0.8017873450750164


Je vais ajouter un L2 regularization pour pénaliser l'overfitting (surapprentissage), car j'ai ajouté des interactions terms. De plus,  L2 regularization encourage les weights à réduire leur magnitude, ce qui va être utile vu que j'ai ajouté des weight au target caravan. Sa devrait rendre le modèle plus stable, mais sans réduire l'importance qu'il doit apporter aux targets.

Je dois trouver la valeure optimale pour lambda pour mon L2. Je vais procéder similairement aux interactions terms.

In [47]:
lambdas = np.logspace(-5, 1, 7)
#score AUC différent vs a compare stats?
best_lambda, best_auc = logisticRegression.find_best_lambda(lambdas, X_train_final_np, y_train_np, X_val_final_np, y_val_np)

Lambda: 1e-05, AUC: 0.8063
Lambda: 0.0001, AUC: 0.8063
Lambda: 0.001, AUC: 0.8063
Lambda: 0.01, AUC: 0.8063
Lambda: 0.1, AUC: 0.8063
Lambda: 1.0, AUC: 0.8063
Lambda: 10.0, AUC: 0.8064
Best lambda: 10.0


<s>Aucune différence pour les différentes valeurs de lambda. Je vais pas ajouter de L2 regularization pour l'instant.</s>

Un petit gain au score AUC pour lambda = 10, que je vais ajouter à mon modèle. La valeur élevé du lambda veut dire que le modèle reste stable et n'overfit pas trop, même avec un L2 regularization.

In [50]:
model = Model.create_model(X_train_final_np, y_train_np, X_val_final_np, y_val_np, learning_rate, extra_weight=ratio, improvement="Ajout du L2 reg", threshold_method="Youden", l2_reg=True, lambda_const=10, to_print=True)
MODEL_L2_BASE = "L2ModelBase"
tracker.add(model, name=MODEL_L2_BASE, set_preferred=True)

Iteration 0: Cost = 0.6931471805599452
Iteration 100: Cost = 0.6078249524424897
Iteration 200: Cost = 0.5990074126769466
Iteration 300: Cost = 0.5946780019276644
Iteration 400: Cost = 0.5919149065856479
Iteration 500: Cost = 0.5899138969613151
Iteration 600: Cost = 0.5883506555685906
Iteration 700: Cost = 0.587066811771201
Iteration 800: Cost = 0.5859750136501103
Iteration 900: Cost = 0.585022745338292
Model L2ModelBase existe deja, overriding...
Model L2ModelBase saved



In [51]:
print(tracker.results)

[{'model': 'v1', 'F1': None, 'threshold': 0.1, 'improvement': ''}, {'model': 'v2', 'F1': None, 'threshold': 0.1, 'improvement': 'adjusted weights'}, {'model': 'v3', 'F1': None, 'threshold': 0.5, 'improvement': 'adjusted weights'}, {'model': 'YoudenModel', 'F1': None, 'threshold': np.float64(0.5287802351530482), 'improvement': 'Youden index threshold'}, {'model': 'inter_terms_1', 'F1': None, 'threshold': np.float64(0.44929478499446074), 'improvement': 'Added interactions terms with auc gain > 0.003'}, {'model': 'inter_terms_2', 'F1': None, 'threshold': np.float64(0.45784582260954015), 'improvement': 'Added interactions terms with auc gain > 0.002'}, {'model': 'L2ModelBase', 'F1': None, 'threshold': np.float64(0.4578161927525435), 'improvement': 'Ajout du L2 reg'}]


In [ ]:
prev_model = tracker.get_by_name(INTER_TERMS_2_MODEL)
logisticRegression.compare_model_stats(X_val_final_np, y_val_np, prev_model, model, isValData=True)

(Avec val data)

Old model :
Threshold:  0.44929478499446074
Accuracy:  0.6412017167381975
[[687 408]
 [ 10  60]]
AUC:  0.8018395303326808

New model :
Threshold:  0.44933251625358467
Accuracy:  0.6412017167381975
[[687 408]
 [ 10  60]]
AUC:  0.8018525766470972


TypeError: object of type 'ModelTracker' has no len()

Pour optimser mon modèle, je vais essayer de trouver un meilleur threshold avec le F1-score, car le F1-score est souvent utilisé pour les datasets imbalanced, donc lui présent.

In [311]:
f1_threshold, score_f1 = logisticRegression.f1_score_threshold(X_val_final_np, y_val_np, model.w ,model.b)
print("f1 threshold : ", f1_threshold)
print("f1 score     : ", score_f1)

f1 threshold :  0.71
f1 score     :  0.31746031746031744


In [312]:
model = Model.Model(model.w, model.b, f1_threshold, score_f1=score_f1, improvement="Added F1 score for finding best threshold")
tracker.add(model, name="FirstF1Model")

Model FirstF1Model saved



In [313]:
prev_model = tracker.get_by_name(MODEL_L2_BASE)
logisticRegression.compare_model_stats(X_val_final_np, y_val_np, prev_model, model)

(Avec training data)

Old model :
Threshold:  0.44929478499446074
Accuracy:  0.6412017167381975
[[687 408]
 [ 10  60]]
AUC:  0.8018395303326808

New model :
Threshold:  0.71
Accuracy:  0.8892703862660944
[[1006   89]
 [  40   30]]
AUC:  0.8018395303326808
F1:  0.31746031746031744


Le modèle est beaucoup plus accurate, mais puisque le threshold est plus élevé, le modèle trouve moins de personne ayant besoin d'assurance caravan. Je vais garder le threshold du F1 score, car il rend le modèle beaucoup plus accurate. Je crois sa serait mieux de continuer de l'optimiser avec ce threshold.

Par contre le F1 score est d'environ 0.27 ce qui est pas très bon, il a encore du mal à prédire les targets.

Puisque j'ai changé au F1 score pour évaluer les performances et choisir le threshold. Je vais revisiter voir si le F1 score aide les interactions terms

In [ ]:
prev_model = tracker.get_by_name(INTER_TERMS_2_MODEL)
model = prev_model.copy()

X_with_terms = data.add_interactions_terms(X_train_interaction_1, interactions_to_add_2)
X_final_with_terms = X_with_terms.to_numpy()

f1_threshold, score_f1 = logisticRegression.f1_score_threshold(X_final_with_terms, y, model.w, model.b)
model.threshold = f1_threshold
model.score_f1 = score_f1
model.improvement = "Interactions terms 2 with F1 threshold"


logisticRegression.compare_model_stats(X_val_final_np, y_val_np, prev_model, model)

(Avec training data)

Old model :
Threshold:  0.4647057775183239
Accuracy:  0.6698161827864628
[[3642 1831]
 [  91  257]]
AUC:  0.7750474639347601

New model :
Threshold:  0.76
Accuracy:  0.907232434289641
[[5177  296]
 [ 244  104]]
AUC:  0.7750474639347601
F1:  0.27807486631016043


In [ ]:
logisticRegression.plot_threshold_metrics(model, X_)

In [ ]:
lambdas = np.logspace(-5, 1, 7)
best_lambda, best_auc = logisticRegression.find_best_lambda(lambdas, X_final_with_terms.to_numpy(), y_data)

Lambda: 1e-05, AUC: 0.7434
Lambda: 0.0001, AUC: 0.7434
Lambda: 0.001, AUC: 0.7434
Lambda: 0.01, AUC: 0.7434
Lambda: 0.1, AUC: 0.7434
Lambda: 1.0, AUC: 0.7434
Lambda: 10.0, AUC: 0.7433
Best lambda: 1e-05


In [ ]:
data.compare_models()

version: 1
Premiere version car 
0.5902765847792476
[[3162 2311]
 [  74  274]]

version: 2
Ajout du traitement des variables discrètes en variables binaires car Certaine variables sont discrètes
0.8739048273492527
[[4944  529]
 [ 205  143]]

version: 3
Traitement de tout les variables en variables binaires car Tout les variables sont discrètes
0.8861020443222813
[[5030  443]
 [ 220  128]]



In [ ]:
data.save_model_state(w,b,threshold,accuracy,conf_matrix,3)